In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
import pandas as pd
import mlflow
import os
import numpy as np

In [2]:
"""
Descarga de Archivos desde Google Drive sin Autenticación

Este script permite descargar archivos específicos desde Google Drive directamente a una ubicación local,
utilizando el ID de Google Drive del archivo (el archivo debe ser accesible para cualquier persona que tenga la URL).

El script procesa un archivo de texto ('archivos_info.txt') que contiene las IDs de Google Drive de los archivos,
los nombres con los que se desean guardar localmente, y las rutas locales de almacenamiento, todo separado por comas.

Formato esperado de 'archivos_info.txt':
ID_de_Google_Drive,Nombre_Archivo_Local,Ruta_Local

Ejemplo:
1hGvKmNAkK...,mi_archivo.txt,./descargas

Requisitos:
- Módulo 'requests' instalado en el entorno Python donde se ejecute este script.



"""

import requests
import os


def descargar_archivo_directo(id_archivo, directorio_destino, archivo_destino):
    """
    Descarga un archivo directamente desde Google Drive y lo guarda localmente.

    Parámetros:
    - id_archivo (str): ID del archivo en Google Drive.
    - directorio_destino (str): Ruta del directorio local donde se guardará el archivo.
    - archivo_destino (str): Nombre deseado para el archivo en local

    Devuelve:
    - archivo_destino (str): Nombre del archivo guardado.
    - ruta_completa (str): Ruta completa del archivo guardado.
    """
    # Construye la URL de descarga directa utilizando el ID del archivo
    url = f"https://drive.google.com/uc?export=download&id={id_archivo}"

    # Realiza la petición HTTP GET para descargar el archivo
    respuesta = requests.get(url, allow_redirects=True)

    # Comprueba que el directorio destino existe, si no, lo crea
    os.makedirs(directorio_destino, exist_ok=True)

    # Construye la ruta completa donde se guardará el archivo en local
    ruta_completa = os.path.join(directorio_destino, archivo_destino)

    # Guarda el contenido del archivo descargado en local
    with open(ruta_completa, 'wb') as archivo:
        archivo.write(respuesta.content)


    return archivo_destino, ruta_completa


def procesar_archivo_info(ruta_archivo_info):
    """
    Procesa un archivo de texto que contiene información sobre los archivos a descargar.

    Parámetros:
    - ruta_archivo_info (str): Ruta del archivo de texto que contiene los IDs de Google Drive,
                               los nombres de los archivos locales y las rutas locales.

    Devuelve:
    - Una lista de tuplas con el ID de Google Drive, el nombre local del archivo, y la ruta local.
    """
    archivos_info = []
    with open(ruta_archivo_info, 'r') as archivo:
        for linea in archivo:
            id_archivo, nombre_archivo, directorio_destino = linea.strip().split(',')
            archivos_info.append((id_archivo, nombre_archivo, directorio_destino))
    return archivos_info


In [3]:
# Ruta al archivo que contiene la información de los archivos a descargar
ruta_archivo_info = "..\info_archivos_GDrive_solo_entr.txt"

# Obtenemos la lista con la info de los archivos del fichero
archivos_a_descargar = procesar_archivo_info(ruta_archivo_info)

# Descargamos cada archivo de la lista
for id_archivo, nombre_archivo, directorio_destino in archivos_a_descargar:
    nombre_archivo_descargado, ruta_archivo_guardado = descargar_archivo_directo(id_archivo, directorio_destino, nombre_archivo)
    print(f"Archivo {nombre_archivo_descargado} guardado en: {ruta_archivo_guardado}")

Archivo datos_preparados_solo_entr.parquet guardado en: ../Downloads\datos_preparados_solo_entr.parquet


In [4]:
df = pd.read_parquet(ruta_archivo_guardado, engine='pyarrow')
df.describe()

,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Tackles,...,nationality_The Gambia,nationality_Türkiye,nationality_Ukraine,nationality_United States,nationality_Uruguay,nationality_Venezuela,nationality_Wales,nationality_Zambia,nationality_Zimbabwe,marketValue
count,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,...,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2.251000e+03
mean,1271.695691,1.564638,1.063527,3.069303,0.168814,0.629276,76.128387,0.946246,0.622834,0.882719,...,0.000444,0.002665,0.001333,0.003554,0.028432,0.003998,0.000888,0.000444,0.000444,8.160329e+06
std,969.241058,2.906800,1.737932,2.958017,0.430942,0.604061,13.308598,0.962542,1.244505,0.693640,...,0.021077,0.051571,0.036490,0.059522,0.166240,0.063119,0.029801,0.021077,0.021077,1.188624e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000e+04
25%,360.500000,0.000000,0.000000,1.000000,0.000000,0.200000,70.950000,0.300000,0.000000,0.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+06
50%,1157.000000,0.000000,0.000000,2.000000,0.000000,0.500000,78.300000,0.700000,0.000000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000e+06
75%,2040.500000,2.000000,2.000000,5.000000,0.000000,0.900000,84.200000,1.300000,1.000000,1.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.500000e+06
max,3420.000000,27.000000,16.000000,15.000000,3.000000,4.500000,100.000000,8.700000,13.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.500000e+07


In [5]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

In [6]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("redes_neuronales")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1713175269356, experiment_id='2', last_update_time=1713175269356, lifecycle_stage='active', name='redes_neuronales', tags={'mlflow.sharedViewState.35fcf56d61b8dbb98512f6829913993e41257533dd332e1e8ff07999f73a21ea': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"1713177259273xowwzda7","type":"BAR","runsCountToCompare":10,"metricSectionId":"171317725927396vp3vfh","deleted":false,"isGenerated":true,"metricKey":"best_mean_test_accuracy"},{"uuid":"1713177259273vnosmd9g","type":"BAR","runsCountToCompare":10,"metri

# Sin Hiperparámetros | Escalado

In [7]:
# Definir listas para almacenar los resultados de entrenamiento y prueba para cada validación cruzada
train_accuracies = []
test_accuracies = []

# Iniciar run de MLflow
with mlflow.start_run(run_name="modelo_completo_sinparam_escalado"):
    # Escalado de características
    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    scalery = StandardScaler()
    y_train_2d = y_train.values.reshape(-1,1)
    scaled_y_train = scalery.fit_transform(y_train_2d)
    y_train_final = scaled_y_train.ravel()

    # Sin definición de hiperparámetros
    clf = MLPRegressor(random_state=RANDOM_STATE)

    # Validación cruzada con recopilación de resultados
    cv_results = cross_validate(clf, scaled_X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)

    # Recopilar los resultados de cada pliegue
    for i, (train_score, test_score) in enumerate(zip(cv_results['train_score'], cv_results['test_score']), 1):
        train_accuracies.append(-train_score)  # Convertir a positivo ya que el scoring es negativo
        test_accuracies.append(-test_score)  # Convertir a positivo ya que el scoring es negativo

        # Guardar los resultados de cada pliegue en MLflow
        mlflow.log_metric(f"train_accuracy_fold_{i}", -train_score)
        mlflow.log_metric(f"test_accuracy_fold_{i}", -test_score)

    # Calcular la media
    mean_train_accuracy = np.mean(train_accuracies)
    mean_test_accuracy = np.mean(test_accuracies)
    
    # Guardar los resultados promedio en MLflow
    mlflow.log_metric("mean_train_accuracy", mean_train_accuracy)
    mlflow.log_metric("mean_test_accuracy", mean_test_accuracy)

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn

# Sin Hiperparámetros | Sin Escalar

In [8]:
# Definir listas para almacenar los resultados de entrenamiento y prueba para cada validación cruzada
train_accuracies = []
test_accuracies = []

# Iniciar run de MLflow
with mlflow.start_run(run_name="modelo_completo_sinparam_noescalado"):
    # Sin definición de hiperparámetros
    clf = MLPRegressor(random_state=RANDOM_STATE)

    # Validación cruzada con recopilación de resultados
    cv_results = cross_validate(clf, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)

    # Recopilar los resultados de cada pliegue
    for i, (train_score, test_score) in enumerate(zip(cv_results['train_score'], cv_results['test_score']), 1):
        train_accuracies.append(-train_score)  # Convertir a positivo ya que el scoring es negativo
        test_accuracies.append(-test_score)  # Convertir a positivo ya que el scoring es negativo

        # Guardar los resultados de cada pliegue en MLflow
        mlflow.log_metric(f"train_accuracy_fold_{i}", -train_score)
        mlflow.log_metric(f"test_accuracy_fold_{i}", -test_score)

    # Calcular la media
    mean_train_accuracy = np.mean(train_accuracies)
    mean_test_accuracy = np.mean(test_accuracies)
    
    # Guardar los resultados promedio en MLflow
    mlflow.log_metric("mean_train_accuracy", mean_train_accuracy)
    mlflow.log_metric("mean_test_accuracy", mean_test_accuracy)

# Con Hiperparámetros | Escalado

In [18]:
param_grid = {
    'hidden_layer_sizes': [(20,), (24,), (28,)],  # Varias configuraciones de capas
    'solver': ['lbfgs', 'sgd', 'adam'], 
    'alpha': [0.0001, 0.001, 0.01],  
    'learning_rate_init' : [0.0001, 0.001, 0.1, 1],
    'batch_size': [16, 32, 64]
}

param_dist = {
    'hidden_layer_sizes': [(x,) for x in range(5, 101, 5)],  # Varias configuraciones de capas
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [10**exp for exp in range(-5, 2)],  # Rango de 10^-5 a 10^1
    'batch_size': [2**exp for exp in range(4, 8)],  # Rango de 16 a 128
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [10**exp for exp in range(-4, 0)],  # Rango de 0.0001 a 0.1
}



# Crear el modelo base
mlp = MLPRegressor(random_state=RANDOM_STATE)

# Configuración de GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=10, n_jobs=-1)

# Configuración de RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=mlp, param_distributions=param_dist, n_iter=10, scoring='neg_mean_absolute_error', cv=10, random_state=RANDOM_STATE, n_jobs=-1)

# Ejecutar Grid Search
with mlflow.start_run(run_name="modelo_completo_grid_escalado"):
    grid_search.fit(scaled_X_train, y_train)
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("grid_search_best_score", -grid_search.best_score_)
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model_grid_search")

# Ejecutar Random Search
with mlflow.start_run(run_name="modelo_completo_randomize_escalado"):
    random_search.fit(scaled_X_train, y_train)
    mlflow.log_params(random_search.best_params_)
    mlflow.log_metric("random_search_best_score", -random_search.best_score_)
    mlflow.sklearn.log_model(random_search.best_estimator_, "model_random_search")

print("Grid Search Best Score: ", -grid_search.best_score_)
print("Random Search Best Score: ", -random_search.best_score_)


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre

Grid Search Best Score:  3216774.278814461
Random Search Best Score:  3157308.535583933


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Con Hiperparámetros | Sin Escalar

In [19]:
param_grid = {
    'hidden_layer_sizes': [(20,), (24,), (28,)],  # Varias configuraciones de capas
    'solver': ['lbfgs', 'sgd', 'adam'], 
    'alpha': [0.0001, 0.001, 0.01],  
    'learning_rate_init' : [0.0001, 0.001, 0.1, 1],
    'batch_size': [16, 32, 64]
}

param_dist = {
    'hidden_layer_sizes': [(x,) for x in range(5, 101, 5)],  # Varias configuraciones de capas
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [10**exp for exp in range(-5, 2)],  # Rango de 10^-5 a 10^1
    'batch_size': [2**exp for exp in range(4, 8)],  # Rango de 16 a 128
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [10**exp for exp in range(-4, 0)],  # Rango de 0.0001 a 0.1
}

# Crear el modelo base
mlp = MLPRegressor(random_state=RANDOM_STATE)

# Configuración de GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=10, n_jobs=-1)

# Configuración de RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=mlp, param_distributions=param_dist, n_iter=10, scoring='neg_mean_absolute_error', cv=10, random_state=RANDOM_STATE, n_jobs=-1)

# Ejecutar Grid Search
with mlflow.start_run(run_name="modelo_completo_grid_escalado"):
    grid_search.fit(X_train, y_train)
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("grid_search_best_score", -grid_search.best_score_)
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model_grid_search")

# Ejecutar Random Search
with mlflow.start_run(run_name="modelo_completo_randomize_escalado"):
    random_search.fit(X_train, y_train)
    mlflow.log_params(random_search.best_params_)
    mlflow.log_metric("random_search_best_score", -random_search.best_score_)
    mlflow.sklearn.log_model(random_search.best_estimator_, "model_random_search")

print("Grid Search Best Score: ", -grid_search.best_score_)
print("Random Search Best Score: ", -random_search.best_score_)


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
13 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
13 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fun

Grid Search Best Score:  2722952.937256958
Random Search Best Score:  2915801.371122822


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
